In [ ]:
import time
import pandas as pd
import numpy as np

In [ ]:
%%time
dataset = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.data', names=["user_id", "item_id", "rating", "timestamp"], sep="\t")
uq_users = np.sort(dataset.user_id.unique().tolist())
uq_items = np.sort(dataset.item_id.unique().tolist())
n_users = len(uq_users)
n_items = len(uq_items)

CPU times: user 39.2 ms, sys: 13.2 ms, total: 52.4 ms
Wall time: 275 ms


In [ ]:
dataset['rating'] /= 5
topk = 10
rank_list = [i+1 for i in range(topk)]
latent = 50

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input, Dot, Dense
from tensorflow.keras import layers, regularizers, optimizers

class GMF(Model):
  def __init__(self, num_users, num_items, latent_dim, regs=[1e-6, 1e-6]):
    super(GMF, self).__init__()
    self.MF_Embedding_User = Embedding(
      input_dim=num_users,
      output_dim=latent_dim,
      name='user_embedding',
      embeddings_initializer='random_uniform',
      embeddings_regularizer=regularizers.l2(regs[0]))
    self.MF_Embedding_Item = Embedding(
      input_dim=num_items,
      output_dim=latent_dim,
      name='item_embedding',
      embeddings_initializer='random_uniform',
      embeddings_regularizer=regularizers.l2(regs[1]))
    self.flatten = Flatten()
    self.predict_vector = Dot(axes=1)
    self.dense = Dense(
      1,
      activation='sigmoid',
      kernel_initializer='lecun_uniform',
      name='prediction')

  @tf.function
  def call(self, inputs):
    # Embedding
    MF_Embedding_User = self.MF_Embedding_User(inputs[0])
    MF_Embedding_Item = self.MF_Embedding_Item(inputs[1])

    # flatten
    user_latent = self.flatten(MF_Embedding_User)
    item_latent = self.flatten(MF_Embedding_Item)

    # Element-wise product of user and item embeddings
    predict_vector = self.predict_vector([user_latent, item_latent])

    # Final prediction layer
    output = self.dense(predict_vector)

    return output

In [ ]:
%%time
model = GMF(n_users, n_items, latent)
model.compile(optimizer=optimizers.Adam(lr=0.01), loss='mean_squared_error')
history = model.fit([dataset.user_id, dataset.item_id], dataset.rating, epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/PartitionedCall:1", shape=(32,), dtype=int64), values=Tensor("Adam/gradients/PartitionedCall:0", shape=(32, 50), dtype=float32), dense_shape=Tensor("Adam/gradients/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/PartitionedCall:4", shape=(32,), dtype=int64), values=Tensor("Adam/gradients/PartitionedCall:3",

3125/3125 [==============================] - 13s 4ms/step - loss: 0.0488
Epoch 2/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0454
Epoch 3/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0440
Epoch 4/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0429
Epoch 5/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0420
Epoch 6/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0417
Epoch 7/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0415
Epoch 8/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0414
Epoch 9/10
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0412
Epoch 10/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0412
CPU times: user 1min 12s, sys: 6.16 s, total: 1min 18s
Wall time: 1min 10s


In [ ]:
%%time
df_recommend_list = pd.DataFrame(columns=['user_id', 'item_id', 'score', 'rank'])
for user_id in uq_users:
  i_list = list(set(uq_items) - set(dataset[dataset['user_id']==user_id]['item_id'].tolist()))
  u_list = [user_id] * len(i_list)
  df_predict = pd.DataFrame()
  df_predict['user_id'] = u_list
  df_predict['item_id'] = i_list
  prediction = model.predict([df_predict.user_id, df_predict.item_id])
  df_predict['score'] = prediction
  df_recommend = df_predict.sort_values('score', ascending=False)[:topk]
  df_recommend['rank'] = rank_list
  df_recommend_list = df_recommend_list.append(df_recommend, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


CPU times: user 1min 47s, sys: 7.42 s, total: 1min 54s
Wall time: 1min 42s


In [ ]:
df_recommend_list

,user_id,item_id,score,rank
0,1,276,0.959656,1
1,1,474,0.951373,2
2,1,430,0.949438,3
3,1,285,0.948800,4
4,1,408,0.948415,5
...,...,...,...,...
9425,943,1177,0.668299,6
9426,943,1176,0.668299,7
9427,943,1175,0.668299,8
9428,943,1174,0.668299,9
